In [23]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1636217982374_0001,pyspark,idle,Link,Link,✔


In [15]:
sc.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 
sc.install_pypi_package("matplotlib", "https://pypi.org/simple") #Install matplotlib from given PyPI repository

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Package already installed for current Spark context!
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1110, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [62]:
sc.install_pypi_package("requests")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


You are using pip version 9.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [76]:
sc.install_pypi_package("pandas")
#sc.uninstall_package('pandas')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

python37-sagemaker-pyspark 1.4.1 requires pyspark==2.3.4, which is not installed.

In [74]:
sc.install_pypi_package("selenium")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached selenium-4.0.0-py3-none-any.whl (954 kB)
  Using cached trio-0.19.0-py3-none-any.whl (356 kB)
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)

In [79]:
sc.install_pypi_package("scipy")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
"""Classes to connect to music databse APIs."""

import base64
import requests

from selenium import webdriver

#from data.utils import rate_limited


class Connection():

    """Base connetion class."""

    def __init__(self, base_url):
        """
        Inialize base connection.

        Args
            base_url: The base url of the API to connect to.
        """
        self.base_url = base_url

    @rate_limited(2)
    def query(self, query):
        """
        Execute an API query.

        query: String containing RESTful API query.
        """
        return requests.get(self.base_url + query)




class WasabiConnection(Connection):

    """Class object instantiates a connection with Wasabi."""

    def __init__(self):
        """Initialize Wasabi API connection."""
        Connection.__init__(self, "https://wasabi.i3s.unice.fr")


class AllMusicConnection(Connection):

    """Class object instantiates a connection with AllMusic.com."""

    def __init__(self, chromedriver_loc):
        """Initialize AllMusic.com connection."""
        Connection.__init__(self, "https://www.allmusic.com")
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--headless")
        self.browser = webdriver.Chrome(
            chromedriver_loc, chrome_options=chrome_options)

    @rate_limited(2)
    def query(self, query):

        self.browser.get(self.base_url + query)


if __name__ == '__main__':

    conn = WasabiConnection()
    r = conn.query("/search/fulltext/Bon%20Jovi")
    print(r)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
import sys
MODULE_FULL_PATH = '/home/notebook/work'
sys.path.insert(1, MODULE_FULL_PATH)

import os
CUR_DIR = os.getcwd()
print("CUR_DIR",CUR_DIR)
#CUR_DIR='/home/notebook/work/data'
CUR_DIR='/home'
dirs = os.listdir(CUR_DIR)
for file in dirs:
   print(file)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

CUR_DIR /
ec2-user
hadoop
emr-notebook

In [83]:
"""Method to rate limit API calls."""

import time
import threading

from functools import wraps
from scipy.sparse import csr_matrix


def rate_limited(max_per_second):
    """
    Rate limit function calls.

    Courtesy of https://gist.github.com/gregburek/1441055
    """
    lock = threading.Lock()
    min_interval = 1.0 / float(max_per_second)

    def decorate(func):
        """Decorate function."""
        last_time_called = [0.0]

        @wraps(func)
        def rate_limited_function(*args, **kwargs):
            """Rate limit function."""
            lock.acquire()
            elapsed = time.time() - last_time_called[0]
            left_to_wait = min_interval - elapsed

            if left_to_wait > 0:
                time.sleep(left_to_wait)

            lock.release()

            ret = func(*args, **kwargs)
            last_time_called[0] = time.time()
            return ret

        return rate_limited_function

    return decorate


def item_user_matrix(X):
    """
    Build the csr matrix of item x user.

    Args
        X: Dataframe with 'item', 'user', 'score' fields.

    Return
        item_user: csr matrix with items as rows, users as columns and score
                   as values.
        item_index: dictionary of the item indexes in the item_user matrix
                    for each item id.
        user_index: dicationary of the user indexes in the item_user matrix
                    for each user_id.

    ---------------------------------------------------------------------------
    """
    X['user_id'] = X['user_id'].astype("category")
    X['song_id'] = X['song_id'].astype("category")

    row = X['song_id'].cat.codes.copy()
    col = X['user_id'].cat.codes.copy()

    nrow = len(X['song_id'].cat.categories)
    ncol = len(X['user_id'].cat.categories)

    item_user = csr_matrix((X['score'], (row, col)), shape=(nrow, ncol))

    user = dict(enumerate(X['user_id'].cat.categories))
    user_index = {u: i for i, u in user.items()}

    item = dict(enumerate(X['song_id'].cat.categories))
    item_index = {s: i for i, s in item.items()}

    return item_user, item_index, user_index


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
"""Downloaders for audio files."""
import pprint
import re
from urllib.parse import quote
from collections import defaultdict

import os
import json
import time
import requests

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from pyspark.shell import spark

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#from data.utils import rate_limited
#from data.connections import WasabiConnection
#from data.connections import AllMusicConnection

import tqdm


class Downloader():

    """Basic Downloader."""

    def __init__(self, save_loc):
        """Initialize downloader."""
        self.save_loc = save_loc

    def download_file(self, url, file):
        """
        Download the specified file to the save_loation directory.

        Args
            url: url where the file is located
            file: the path of the file to save (including the extention)
        """
        file_name = os.path.join(self.save_loc, file)
        print(file)
        print(file_name)
        print(os.path.isfile(file))
        if not os.path.isfile(file):
            print("downloadfile ", url)
            print("requesting")
            r = requests.get(url)
            print("done")
            print("Saving {}".format(file_name))
            print(file)

            with open(file, "wb") as f:
                f.write(r.content)

    def save_json(self, obj, file):
        """
        Save object to the specified path in json format.

        Args
            obj: an object to save as json
            file: the path of the file to save (including the extention)
        """
        file_name = os.path.join(self.save_loc, file)

        if not os.path.isfile(file):
            with open(file, "w") as f:
                print("Saving {}.json".format(file_name))
                json.dump(obj, f)


class WasabiDownloader(Downloader):

    """Downloader for the Wasabi music database."""

    def __init__(self, save_loc, found_dir, notfound_dir):
        """
        Initialize Downloader and crate connection to Wasabi API.

        Args
            save_loc: Directory that contains the folders found_dir and
                      notfound_dir where the json and mp3 files will be saved.
            found_dir: Directory name where found song data is saved.
            notfound_dir: Directory name where not found song data is saved.
        """
        print("WasabiDownloader")
        Downloader.__init__(self, save_loc)
        self.conn = WasabiConnection()
        self.save_loc = save_loc
        self.found_loc = os.path.join(save_loc, found_dir)
        self.notfound_loc = os.path.join(save_loc, notfound_dir)

    @rate_limited(1)
    def _search_artist(self, artist):
        """
        Use the Wasabi API to search for the artistself.

        Finds the best matching result and uses this artist. Rate limited to
        @rate_limit(x)

        Args
            artist: name of the artist.

        Return
            Best matching result on Wasabi database for the given name of
            artist.

        -----------------------------------------------------------------------
        """
        # format artist name according to REST API requirements and make url
        artist = quote(artist, safe='')
        r = self.conn.query("/search/fulltext/"+artist)

        if r.content.find(b'Error') > -1:
            return None
        else:
            r = r.json()

        # return the best matching result found in "namSuggest" field
        if r and 'nameSuggest' in r[0]:
            best_match = r[0]['nameSuggest']['input'][0]
            return best_match

        return None

    @rate_limited(1)
    def _get_artist_data(self, artist):
        """
        Use the Wasabi API to return the artist data provided by Wasabiself.

        For an *exact* artist name that *exists* in the Wasabi database.
        Rate limited to @rate_limit(x)

        Args
            artist: exact artist name that exists in the Wasabi database.

        Return
            artist data json return from the Wasabi API.

        -----------------------------------------------------------------------
        """
        # format artist name according to REST API requirements and make url
        artist = quote(artist, safe='')
        r = self.conn.query("/search/artist/"+artist)
        print("r ",r)
        artist_data = r.json()

        if 'error' in artist_data:
            return None

        return artist_data

    @staticmethod
    def _get_artist_songs(artist_data):
        """
        Collect the songs from a given artist data json.

        Args
            artist_data: artist data json return from the Wasabi API.

        Return
            dataframe of artist, album and songs.

        -----------------------------------------------------------------------
        """
        albums = artist_data['albums']
        songs = defaultdict(list)
        for album in albums:
            for song in album['songs']:

                songs['artist_id'] += [artist_data['_id']]
                songs['artist'] += [artist_data['name']]
                songs['album'] += [album['title']]
                songs['album_id'] += [album['_id']]
                songs['title_lower'] += [str(song['title']).lower()]

                for k, v in song.items():
                    songs[k] += [v]

        return pd.DataFrame(songs)

    @rate_limited(2)
    def _get_song_data(self, song_id):
        """
        Use a Wasabi song id to retun the song data from the Wasabi API.

        Args
            wasabi_song_id: a song id from the Wasabi API.

        Return
            song data json returned from the Wasabi API.

        -----------------------------------------------------------------------
        """
        # call the rest api and convert to json
        success = False
        while success is False:
            r = self.conn.query("/api/v1/song/id/"+song_id)
            if r.content == b'Too many requests, please try again later.':
                success = False
                time.sleep(20)
            else:
                success = True
                song_data = r.json()

        return song_data

    def scrape(self, track_df):
        """
        Download previews and save song data json for tracks from track_df.

        Args
            track_df: Dataframe of the tracks that should be processed.  Must
                      contain the fields "artist", "title_lower", and
                      "song_id".
            save_loc: Directory that contains the folders "clips" and "noclips"
                      where the json and mp3 files will be saved.

        Return
            saves the song data json and mp3 files for previews.

        -----------------------------------------------------------------------
        """
        # artists = [str(row.artist_name) for row in track_df.select("artist_name").collect()]
        track_df = track_df.withColumnRenamed("artist_name","artist").distinct()
        track_df = track_df.toPandas()

        def run(string):

            # Make own character set and pass
            # this as argument in compile method
            regex = re.compile('[@_!#$%^&*()<>?/\|}{~:]')

            # Pass the string in search
            # method of regex object.
            if (regex.search(string) == None):
                return string

            else:
                return None

        artists = np.sort(track_df['artist'].dropna().unique())
        artists = [run(artist) for artist in artists]
        artists = list(filter(None, artists))

        print("artists:", len(artists))
        # search each artist
        for artist in artists:
            artist_data = None
            intersect = None
            print("starting artist ", artist)
            artist = self._search_artist(artist.lower())
            print("artist found from api ", artist)
            msd_songs = track_df[track_df['artist'] == artist]
            if not msd_songs.empty:
                msd_songs_df = spark.createDataFrame(msd_songs)
                msd_songs_df.show()
            if artist is not None:
                artist_data = self._get_artist_data(artist)
                print("found artist data ",artist_data)

            if artist_data is not None:
                songs_df = spark.createDataFrame(self._get_artist_songs(artist_data))
                songs_df.show(10,False)
                intersect = msd_songs.merge(
                    self._get_artist_songs(artist_data),
                    on=['title_lower']).dropna()
                if not intersect.empty:
                    intersect_df = spark.createDataFrame(intersect)
                    intersect_df.show(10,False)

            # add to track dict for each song in both data sets

            if intersect is not None and intersect.shape[0] > 0:
                print("here")
                print(len(intersect))

                for _, row in intersect.iterrows():

                    mp3_file = "{}.mp3".format(row['song_id'])
                    mp3_exists = os.path.isfile(
                        os.path.join(self.found_loc, mp3_file))

                    json_file = "{}.json".format(row['song_id'])
                    json_exists = os.path.isfile(
                        os.path.join(self.found_loc, json_file))

                    track = self._get_song_data(row['_id'])
                    track['song_id'] = row['song_id']

                    if track['preview']:
                        self.save_loc = self.found_loc
                        if not mp3_exists:
                            self.download_file(track['preview'], mp3_file)
                        if not json_exists:
                            self.save_json(track, json_file)
                    else:
                        self.save_loc = self.notfound_loc
                        print("No preview for {} by {}".
                              format(track['title'], artist))
                        if not json_exists:
                            self.save_json(track, json_file)

    def _get_songs_byindex(self, start_index):
        """
        Use a start index to retun the song data from the Wasabi API.

        Args
            wasabi_song_id: a song id from the Wasabi API.

        Return
            song data json returned from the Wasabi API.

        -----------------------------------------------------------------------
        """
        # call the rest api and convert to json
        success = False
        while success is False:
            r = self.conn.query("/api/v1/song_all/"+str(start_index))
            if r.content == b'Too many requests, please try again later.':
                success = False
                time.sleep(20)
            else:
                success = True
                song_data = r.json()

        return song_data


class AllMusicDownloader(Downloader):

    """Downloader for AllMusic.com."""

    def __init__(self, save_loc, chromedriver_loc):
        """
        Initialize the downloader.

        Args
            save_loc: location to save the downloaded bios.
            chromedriver_loc: path to the chromedriver.exe
        """
        Downloader.__init__(self, save_loc)
        self.conn = AllMusicConnection(chromedriver_loc)
        self.save_loc = save_loc

    def _goto_artist_from_song(self, song_query):
        """
        Navigate to the artist page from a song page string.

        Args
            song_query: the last part of the song page url.
        """
        self.conn.query("/song/"+quote(song_query, safe=''))
        WebDriverWait(self.conn.browser, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "song-artist")))
        soup = BeautifulSoup(self.conn.browser.page_source, 'html.parser')
        soup = BeautifulSoup(soup.decode_contents(), 'lxml')
        soup = soup.find("h2", {"class": "song-artist"})
        artist_link = soup.find("a")["href"]
        self.conn.browser.get(artist_link)

    def _soup_artist_page(self):
        """
        Soup the artist page.

        Return
            Soup of the artist page

        -----------------------------------------------------------------------
        """
        soup = BeautifulSoup(self.conn.browser.page_source, 'html.parser')
        soup = BeautifulSoup(soup.decode_contents(), 'lxml')
        return soup

    @staticmethod
    def _get_artist_shortbio(artist_page_soup):
        """
        Extract the artist short bio.

        Args
            artist_page_soup: soup of the artist page html.

        Return
            (string) artist short bio.

        -----------------------------------------------------------------------
        """
        return artist_page_soup.find("span", {"itemprop": "reviewBody"}).text

    def _get_artist_longbio(self, artist_page_soup):
        """
        Extract the artist short bio.

        Args
            artist_page_soup: soup of the artist page html.

        Return
            (list of strings) each list element corresponds to a paragraph.

        -----------------------------------------------------------------------
        """
        bio_link = artist_page_soup.find(
            "p", {"class": "biography"}).find("a")["href"]
        self.conn.browser.get(bio_link)
        WebDriverWait(self.conn.browser, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "text")))
        soup = BeautifulSoup(self.conn.browser.page_source, 'html.parser')
        soup = BeautifulSoup(soup.decode_contents(), 'lxml')
        soup = soup.find("div", {"itemprop": "reviewBody"})
        long_bio = [x.strip() for x in soup.text.split("\n")
                    if (len(x.strip()) > 0 and x.find("<") == -1)]
        return long_bio

    def scrape(self, songlinks_df):
        """
        Scrape to collect artist short and long bios.

        Args
            songlinks_df: a dataframe with columns 'song_id' and 'url_allmusic'

        Return
            (dataframe) saves original dataframe with short and long bios.

        -----------------------------------------------------------------------
        """
        total = songlinks_df.shape[0]
        progress = tqdm.tqdm(total=total)
        short_bios = []
        long_bios = []
        for _, row in songlinks_df.iterrows():
            try:
                song_url = row['url_allmusic'].split("/")[-1]
                song_id = row['song_id']
                try:
                    self._goto_artist_from_song(song_url)
                    artist_page_soup = self._soup_artist_page()
                    short_bios += [self._get_artist_shortbio(artist_page_soup)]
                    long_bios += [self._get_artist_longbio(artist_page_soup)]
                except Exception:
                    print("Did not process {}".format(song_id))
                    short_bios += [None]
                    long_bios += [None]
                progress.update(1)
            except Exception as e:
                print(e)
                print("Saving current progress...")
                songlinks_df['short_bio'] = short_bios + \
                    [None]*(total-len(short_bios))
                songlinks_df['long_bio'] = long_bios + \
                    [None]*(total-len(long_bios))
                songlinks_df.to_csv(self.save_loc, index=False)
                self.conn.browser.quit()

        songlinks_df['short_bio'] = short_bios
        songlinks_df['long_bio'] = long_bios
        songlinks_df.to_csv(self.save_loc, index=False)
        self.conn.browser.quit()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
"""Script to download audio files."""

import os
import glob

import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import lower, col, regexp_replace
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

#from data.downloaders import WasabiDownloader


def get_visited_song_ids(msd_dir):
    """
    Build a list of the previously processed MSD song idsself.

    Used so that data collection can resume if it is interrupted.

    Args
        msd_dir: Directory that contains the clips and the noclips folders
        where the mp3 and song data json are saved.

    Return
        list of the MSD song ids that have already been processed.

    ---------------------------------------------------------------------------
    """
    ids = []
    for f in glob.glob(os.path.join(msd_dir, "clips", "*.json")):
        ids += [f.split("/")[-1].split(".")[0]]

    for f in glob.glob(os.path.join(msd_dir, "noclips", "*.json")):
        ids += [f.split("/")[-1].split(".")[0]]

    return ids


#if __name__ == "__main__":

spark = SparkSession \
    .builder \
    .appName("project") \
    .config("...") \
    .getOrCreate()

CUR_DIR = os.getcwd()

file = os.path.join(CUR_DIR, "data", "unique_tracks.txt")


# load taste profile dataset
file = os.path.join(CUR_DIR, "data", "train_triplets1.txt")
colnames = ['user_id', 'song_id', 'playcount']
schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("song_id", StringType(), True),
    StructField("playcount", IntegerType(), True)])
taste_df = spark.read.csv(file, header=False,sep='\t',schema=schema)
taste_df.printSchema()
# load msd track info
file = os.path.join(CUR_DIR, "data", "unique_tracks.txt")

track_df = spark.read.csv(file, header=True,sep=',').withColumnRenamed("song_id","songid")
str_cols = [f.name for f in track_df.schema.fields if isinstance(f.dataType, StringType)]
print(str_cols)
for c in str_cols:
    track_df = track_df.withColumn(c, regexp_replace(col(c), '"', ""));
    track_df = track_df.withColumn(c, regexp_replace(col(c),  "'", ""));

track_df.select(*str_cols).show(10,False)

select_cols = ['song_id', 'track_id', 'artist_name', 'song_title']
taste_df = taste_df.join(track_df, taste_df["song_id"] == track_df["songid"], "inner").select(*select_cols)
taste_df.show()

taste_df = taste_df.withColumn('title_lower', lower(col('song_title')))

save_loc = os.path.join(CUR_DIR, "audio", "MSD")
wasabi_downloader = WasabiDownloader(save_loc, 'clips', 'noclips')
wasabi_downloader.scrape(taste_df)
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'Path does not exist: hdfs://ip-172-31-50-116.ec2.internal:8020/data/train_triplets1.txt;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 476, in csv
    return self._df(self._jreader.csv(self._spark._sc._jvm.PythonUtils.toSeq(path)))
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'Path does not exist: hdfs://ip-172-31-50-116.ec2.internal:8020/data/train_triplets1.txt;'

